In [18]:
#;.pykx.disableJupyter()

PyKX now running in 'python' mode (default). All cells by default will be run as python code. 
Include '%%q' at the beginning of each cell to run as q code. 


In [19]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


# Scripting And Logging

**Learning Objectives**

To understand 


+ How to create a q script

+ The ways to load a q script

+ How to implement Multi-line Comments

+ How to pass arguments to a script

+ How to implement logging

# Introduction

A script is a program or sequence of instructions which are interpreted by another program before execution of each instruction.

In kdb+/q, we can interactively execute q commands in our command prompt:

In [17]:
a:10
b:3.2
fun : {x+y}
\v
\f

`s#`a`b`x`y
,`fun


Here we have defined some variables and a function, however if we close our q session and start another q session (so here, that would be restarting the notebook kernel), we lose these variables and functions.

In [15]:
a

10


Instead of manually executing instructions for each instance, scripting allows us to automate the execution of elaborate instruction sets as a program; a process that is significantly faster and less prone to error than with human interaction.

# Creating q scripts

To create a q script, go to the 

    Jupyter Notebook Home Page->08 Scripting and Logging->New->Text File
Give a name - "qscript.q" to this text file. **Take care to include the ".q" suffix**

Paste the following lines in the text(.q) file - 

    a : 25;
    lambda : {x * y};
    0N!"Hello";
    
We (but really you) just created a q script! 

## Commenting in a script 

When writing a script to perform a particular  task, it's important to include appropriate comments so the script can be maintained in the future. We have seen comments throughout the material so far - these are indicated with a leading `/` and can be placed on their own line or in-line with some code. 

In [18]:
//Beginning generation
x: 10?20.0 
avg x //As per JIRA 23.1 - this is an in-line comment 

12.12923


Commenting style is somewhat of a personal preference, but the author uses individual line comments to denote new logical separations in the code, and in-line to explain particular lines of code.

Scripts allow us to write comments that span multiple lines - something we cannot do from the command prompt of a running q session.

Open the comment block with a single forward slash. Close it with a single backward slash. 

In [19]:
/
  This is a comment block.
  Q ignores everything in it.

  And I mean everything.
  2+2
\

The use of the double `//` for individual line comments by the author is also just personal style - this  is a personal choice to differentiate between block comments and single line comments but in actuality `/` would work just as well.

# Running a q script 

If we wanted to start a new q process and have it begin by running this script we can start our q process with the script name - e.g. 
    
    mycomputerprompt$ q qscript.q

In our case, this notebook is already a running q process, so we need to load the script *interactively*. We can load the script in to our q session (interactively) using one of the load file system commands.

In [22]:
\v
\f

`s#`a`b`x`y
`s#`fun`lambda


In [23]:
\l qscript.q           //l for load
//system "l qscript.q"

"Hello"


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Using the <code>system</code> command in kdb+/q sends a message for execution to the operating system. We can use this to execute more generic system commands when needed.  
</i></p>

In [24]:
system "pwd"
\pwd

"/home/jovyan/course-fundamentals/08 Scripting And Logging"
"/home/jovyan/course-fundamentals/08 Scripting And Logging"


## Calling a script from another script

We can create a script which is called from another script. Below, we have created a file `masterscript.q` which contains the following - 

<code>
0N!"Hello, This is the master script";
\l secondaryscript.q
</code>

This loads another script called `secondaryscript.q` which contains the following line of code - 

<code>0N!"This is the secondary script";</code>

In [25]:
\l masterscript.q

"Hello, This is the master script"
"This is the secondary script"


Both the scripts have been executed.

# Passing parameters to a script

Parameter variables can be used by scripts to influence the output of the script, which can be defined within the interactive session prior to script execution, or passed to a script dynamically from the command line.

Here we add the following line to our script to say hello to a single person -

     0N!"Hello",name;

In [4]:
name : "Arsene"
\l qscript.q

"Hello Arsene"


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>It is generally recommended to avoid writing scripts which are dependent on local variables being manually declared and existing in the calling q process. Better practice is to either store these in configuration files (which themselves are loaded as part of the script) or we can pass parameters to our q script at time of calling.
</i></p>

## Command line parameters - `.z.x` 
A better approach using the command line would be to pass in the variable and have our script parse the value:


Replace the Hello name line with this in qscript.q-

    0N!("The value of .z.x is -");
    0N!.z.x;
    0N!"hello ",.z.x[1];
    0N!"Exiting the script";
    exit 0;

Here we see the introduction of [`.z.x`](https://code.kx.com/q/ref/dotz/#zx-raw-command-line) which captures the command line arguments as a list of strings.

In [8]:
\env QHOME=/opt/kx/q q qscript.q -name Arsene

"\"hello Arsene\""
"\"Exiting the script\""


.z.x simply captures the arguments without formatting them. We would need to provide the index to retrieve a specific parameter. Could there be a better way to do this?

In [6]:
system["env QHOME=/opt/kx/q q qscript.q -name Arsene -age 56 abc xyz"]

"\"The value of .z.x is -\""
"(\"-name\";\"Arsene\";\"-age\";\"56\";\"abc\";\"xyz\")"
"\"hello Arsene\""
"\"Exiting the script\""


## Formatting Command line parameters - `.Q.opt`

[`.Q.opt`](https://code.kx.com/q/ref/dotq/#qopt-command-parameters) takes the list of command line arguments and builds a dictionary for us to easily access the command line parameters using key/value pairs which we can assign to a variable `d`:

We can view the dictionary by adding `show d` to the code in the text file.

In `qscript.q`, let's replace the lines changed above with the following - 

    0N!("The value of .z.x is -");
    0N!.z.x;
    d:.Q.opt .z.x;
    0N!"The value of .Q.opt .z.x is -";
    show d;
    0N!"hello ",first d[`name];
    0N!"Exiting the script";
    exit 0;

In [9]:
system["env QHOME=/opt/kx/q q qscript.q -name Arsene "]

"\"The value of .z.x is -\""
"(\"-name\";\"Arsene\")"
"\"The value of .Q.opt .z.x is -\""
"name| \"Arsene\""
"\"hello Arsene\""
"\"Exiting the script\""


Let's see what happens when we add more command-line arguments.

In [10]:
system["env QHOME=/opt/kx/q q qscript.q -name Arsene -age 56"]

"\"The value of .z.x is -\""
"(\"-name\";\"Arsene\";\"-age\";\"56\")"
"\"The value of .Q.opt .z.x is -\""
"name| \"Arsene\""
"age | \"56\"    "
"\"hello Arsene\""
"\"Exiting the script\""


**Exercise**


Create a script which takes 4 numbers (20.7;12.8;40;11.5) as command-line arguments and returns the sum of the numbers.

In [13]:
// Your Answer - Load the script here
0N!("The value of .z.x is -");
0N!.z.x;
d:.Q.opt .z.x;
0N!"The value of .Q.opt .z.x is -";
show d;
d : {-9h$x} each d;
-1 "The sum of the arguments is -";
show sum value d;



"The value of .z.x is -"
()
"The value of .Q.opt .z.x is -"
The sum of the arguments is -


ex1.q

    0N!("The value of .z.x is -");
    0N!.z.x;
    d:.Q.opt .z.x;
    0N!"The value of .Q.opt .z.x is -";
    show d;
    d : {-9h$x} each d;
    -1 "The sum of the arguments is -";
    show sum value d;


In [14]:
system["env QHOME=/opt/kx/q q ex1.q -num1 20.7 -num2 12.8 -num3 40 -num4 11.5"]

'2025.02.25T04:27:39.549 ex1.q. OS reports: No such file or directory


QError: os

# Adding logging to scripts

It is generally good practice to log as much detail as possible in scripts for reference and debugging.

A function to log normal messages to stdout (-1) and error message to stderr (-2) should be defined. See [File Handles](https://code.kx.com/v2/basics/files/?_ga=2.142078130.1692917060.1595324408-240122142.1591882105) for more info.

In [15]:
out:{-1 string[.z.p]," ### INFO ### ",x};
err:{-2 string[.z.p]," ### ERROR ### ",x};


sayhello:{out["Hello ",x]};

sayhello["Nicolas"]

-1
2025.02.25D05:03:35.783829380 ### INFO ### Hello Nicolas


This will return an error. We'll have to restart the notebook kernel after running this.

In [16]:
@[sayhello;"Martin";{err "Error running main: ",x;exit 1}]; 

2025.02.25D05:04:01.239217703 ### INFO ### Hello Martin


Another example of logging. The qlogexample.q file is used for the below example. The content of qlogexample.q is as follows - 


    out:{-1 string[.z.p]," ### INFO ### ",x};
    err:{-2 string[.z.p]," ### ERROR ### ",x};

    sayhello:{out["Hello ",x]};
    sayhello["Nicolas"];

    d:.Q.opt .z.x;
    if[0=count d; out["No parameter provided"]];

    0N!"hello ",first d[\`name];
    exit 0;


We can redirect the output to a logging file using the below command

In [17]:
\nohup env QHOME=/opt/kx/q q qlogexample.q >>myLog.txt 2>&1 &

()


A file named myLog.txt is created in the default directory. The logs are stored in that file.


In [20]:
system["env QHOME=/opt/kx/q q qlogexample.q"] //Without providing parameters

"2025.02.25D05:09:48.846782411 ### INFO ### Hello Nicolas"
"2025.02.25D05:09:48.846855792 ### INFO ### No parameter provided"
"\"hello \""


In [21]:
system["env QHOME=/opt/kx/q q qlogexample.q -name Pierre"] // providing the name parameter

"2025.02.25D05:09:56.053811552 ### INFO ### Hello Nicolas"
"\"hello Pierre\""


In [22]:
system["env QHOME=/opt/kx/q q qlogexample.q > log1.txt"] //This will redirect the output to a file log1.txt

"2025.02.25D05:10:02.014170332 ### INFO ### Hello Nicolas"
"2025.02.25D05:10:02.014288292 ### INFO ### No parameter provided"
"\"hello \""


Consider the following script (qlogexample2.q) -

<code>
    out:{-1 string[.z.p]," ### INFO ### ",x};
    err:{-2 string[.z.p]," ### ERROR ### ",x};
    printProduct : {out["The product is = ",string x*y]};
    d:.Q.opt .z.x;
    0N!d;
    a:"J"$first first d[`num1];
    b:"J"$first first d[`num2];
    if[0=count d; err["No parameter provided"]];
    $[(0N="J"$first first d[`num1]) or (0N="J"$first first d[`num2]); err["Type Mismatch : Please provide a long integer value"]; printProduct[a;b]] ;
    exit 0;
</code>

This will multiply two integers. However, it will return an error if the parameters provided are not numbers.

In [23]:
system["nohup env QHOME=/opt/kx/q veq qlogexample2.q -num1 3 -num2 4 >myLog2.txt 2>&1 &"]

()


Check the contents of myLog2.txt after execution. Now let's deliberately put a random text in num2.


In [24]:
system["nohup env QHOME=/opt/kx/q q qlogexample2.q -num1 3 -num2 abc >myLog2.txt 2>&1 &"]

()


Check the contents of myLog2.txt after execution. There should be an error message printed using stderr.

Now, lets see how we can use protected evaluation in a script to catch errors.
Consider the following script to print the `til` list for an integer.
qlogexample3.q

<code>
    out:{-1 string[.z.p]," ### INFO ### ",x};
    err:{-2 string[.z.p]," ### ERROR ### ",x};
    printList : {0N!til x};
    d:.Q.opt .z.x;
    0N!d;
    a:"J"$first first d[`num1];
    b:"J"$first first d[`num2];
    if[0=count d; err["No parameter provided"]];
    0N!a;
    0N!b;
    @[printList;a;{err "Error running main: ",x;exit 1}];
    @[printList;b;{err "Error running main: ",x;exit 1}];
    exit 0;
</code>

In [25]:
system["nohup env QHOME=/opt/kx/q q qlogexample3.q -num1 3 -num2 abc >myLog3.txt 2>&1 &"]

()


When we open myLog3.txt, we find that the first list was printed correctly whereas the second execution resulted in an error which was caught using [`@`](https://code.kx.com/q4m3/10_Execution_Control/#1018-protected-evaluation).

**Exercise**

Write a script(ex2.q) which has functions for printing logs to stderr and stdout, and uses protected evaluation to multiply two lists. Create these 3 lists in the script -

    x: 14 16 17 74 4 
    y: 0 1 1 0 4
    z: 2 -1 2 1
    
Now, call the function with x;y and y;z as parameters and redirect the output to a log file.

In [ ]:

out:{-1 string[.z.p]," ### INFO ### ",x};
err:{-2 string[.z.p]," ### ERROR ### ",x};
func : {[a;b] 
 .[*;(a;b);{err "Error running main: ",x;exit 1}];
  out "the answer is = ", " " sv string *[a;b];
 }
x: 14 16 17 74 4;
y: 0 1 1 0 4;
z: 2 -1 2 1;
func[x;y];
func[y;z];
exit 0;

In [ ]:
system["nohup env QHOME=/opt/kx/q q ex2.q >logex2.txt 2>&1 &"]

e2.q

    out:{-1 string[.z.p]," ### INFO ### ",x};
    err:{-2 string[.z.p]," ### ERROR ### ",x};
    func : {[a;b] 
     .[*;(a;b);{err "Error running main: ",x;exit 1}];
      out "the answer is = ", " " sv string *[a;b];
     }
    x: 14 16 17 74 4;
    y: 0 1 1 0 4;
    z: 2 -1 2 1;
    func[x;y];
    func[y;z];
    exit 0;